In [1]:
TESTING_SUBSET = True  # ***** FOR TESTING ONLY !!!!!!!! ******

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:

    # Enable line-wrapping
    # from IPython.display import HTML, display

    # def set_css():
    #     display(HTML('''
    #         <style>
    #         pre {
    #             white-space: pre-wrap;
    #             }
    #         </style>
    #     '''))
    # get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

    %pip install transformers
    %pip install datasets
    %pip install pynvml

In [3]:
if IN_COLAB:
    # Enable line-wrapping
    from IPython.display import HTML, display

    def set_css():
        display(HTML('''
            <style>
            pre {
                white-space: pre-wrap;
                }
            </style>
        '''))
    get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

In [4]:
import datetime
import numpy as np
from sklearn.metrics import fbeta_score
from matplotlib import pyplot as plt
import math
import pandas as pd
import sys
import json
import time
import torch
from torch import nn
import random
#from tqdm.auto import tqdm
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import Dataset
import os
import torchsummary
from functools import partialmethod
from itertools import product

from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass
from transformers import PreTrainedTokenizer
from torch.utils.data import DataLoader

if IN_COLAB:
  sys.path.append('/content/drive/MyDrive/ML/lecr/code/')
from l_utils import *
from model_utils import *

#from model_defs import Topic_NN, Content_NN

%env CUBLAS_WORKSPACE_CONFIG=:16:8

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

/home/mahesh/Desktop/ML/kaggle/LearningEquality/learning-equality-code/le-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-30 09:56:55.930871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 09:56:56.070442: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2023-03-30 09:56:56.535051: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-03-30 09:56:56.535301: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-03-30 09:56:56.535309: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


env: CUBLAS_WORKSPACE_CONFIG=:16:8
cuda


In [5]:
if IN_COLAB:
    class CFG:
        DIRNAME_PROCESSED_DATA = '/content/drive/MyDrive/ML/lecr/data/'
        FILENAME_DATA_CONTENT_F = DIRNAME_PROCESSED_DATA + "content.f"
        FILENAME_DATA_TOPICS_F = DIRNAME_PROCESSED_DATA + "topics.f"
        FILENAME_DATA_CORRELATIONS_CSV = DIRNAME_PROCESSED_DATA + "correlations.csv"
        NUM_FOLDS = 5
        SAVE_MODEL_DIR = '/content/drive/MyDrive/ML/lecr/models/'
else:
    from cfg import *

# TRANS_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
TRANS_MODEL = "sentence-transformers/distiluse-base-multilingual-cased-v2"
topics_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "topics_tokenized_ml64_dist.pq"
contents_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "contents_tokenized_ml64_dist.pq"


In [6]:
!nvidia-smi

Thu Mar 30 09:56:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   54C    P2    61W / 275W |   5242MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# Split data into folds

# Feature engineering ... use same as the one used for bce-lightning

# Training for a Top-k metric ... see if this performs better than directly using minilm embeddings

In [8]:
def progress_bar_control(show_progress_bars = True):
    tqdm.__init__ = partialmethod(tqdm.__init__, disable=(not show_progress_bars))

In [9]:
contents_df = pd.read_feather(CFG.FILENAME_DATA_CONTENT_F)
topics_df   = pd.read_feather(CFG.FILENAME_DATA_TOPICS_F)
correlations_df =  pd.read_csv(CFG.FILENAME_DATA_CORRELATIONS_CSV)

In [10]:
contents_df.info()
topics_df.info()
correlations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154047 entries, 0 to 154046
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                154047 non-null  object
 1   title             154038 non-null  object
 2   description       89456 non-null   object
 3   kind              154047 non-null  object
 4   text              74035 non-null   object
 5   language          154047 non-null  object
 6   copyright_holder  71821 non-null   object
 7   license           74035 non-null   object
dtypes: object(8)
memory usage: 9.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76972 entries, 0 to 76971
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           76972 non-null  object
 1   title        76970 non-null  object
 2   description  34953 non-null  object
 3   channel      76972 non-null  object
 4   category     76972 no

In [11]:
topics_tokenized_df   = pd.read_parquet(topics_tokenized_pq_file)
contents_tokenized_df = pd.read_parquet(contents_tokenized_pq_file)

In [12]:
contents_tokenized_df.info()
topics_tokenized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154047 entries, 0 to 154046
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id                  154047 non-null  object
 1   c_0_input_ids       154047 non-null  object
 2   c_1_attention_mask  154047 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61517 entries, 0 to 61516
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  61517 non-null  object
 1   t_0_input_ids       61517 non-null  object
 2   t_1_attention_mask  61517 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [13]:
# train_df_orig, val_df_orig = create_train_val_split_pos_corr(topics_df, contents_df, correlations_df, CFG.NUM_FOLDS, min_train_perc = 80.0, use_topic_trees = True, random_seed = 3) # pick a random seed that gives a good split
train_df_orig, val_df_orig = create_train_val_split_pos_corr_from_kaggle(topics_df, correlations_df, num_splits = 4, val_fold = 0)

177290
102629
2023-03-30 09:57:00.585116 : Splitting non-source into 4 folds
fold
0.0    25667
1.0    25677
2.0    25663
3.0    25622
dtype: int64
fold
0.0    6070
1.0    4971
2.0    8493
3.0    5469
dtype: int64


In [14]:
# ***** FOR TESTING ONLY !!!!!!!! ******
if TESTING_SUBSET:
    train_df = train_df_orig[:1000]
    val_df = val_df_orig[:1000]
else:
    train_df = train_df_orig
    val_df = val_df_orig

In [15]:
train_df.info()
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   t_id    1000 non-null   object
 1   c_id    1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   t_id    1000 non-null   object
 1   c_id    1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [16]:
#train_df.loc[random.sample(range(0,len(train_df)),10)]
train_df.loc[train_df['t_id']=='t_f2872cd7562c']

,t_id,c_id


In [17]:
PARTITION_BATCH_SIZE = 350

In [18]:
train_tokenized_df = get_partitioned_data_for_pos_corr(train_df, topics_tokenized_df, contents_tokenized_df, batch_size = PARTITION_BATCH_SIZE, do_partitioning = False, sort = False)
# train_tokenized_df = get_labeled_data_for_pos_corr(train_df.copy(), topics_tokenized_df, contents_tokenized_df)

2023-03-30 09:57:00.755972 : Merging in topic and content data into the folds data ... 


In [19]:
train_tokenized_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   t_0_input_ids       1000 non-null   object
 1   t_1_attention_mask  1000 non-null   object
 2   c_0_input_ids       1000 non-null   object
 3   c_1_attention_mask  1000 non-null   object
dtypes: object(4)
memory usage: 39.1+ KB


In [20]:
val_topics_tokenized_df, val_contents_tokenized_df, val_corr_df = get_pos_corr_subsets_for_binary_fold( val_df, contents_tokenized_df, topics_tokenized_df)

In [21]:
val_topics_tokenized_df.info()
val_contents_tokenized_df.info()
val_corr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   index               239 non-null    int64 
 1   id                  239 non-null    object
 2   t_0_input_ids       239 non-null    object
 3   t_1_attention_mask  239 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   index               984 non-null    int64 
 1   id                  984 non-null    object
 2   c_0_input_ids       984 non-null    object
 3   c_1_attention_mask  984 non-null    object
dtypes: int64(1), object(3)
memory usage: 30.9+ KB
<class 'pandas.core.series.Series'>
Int64Index: 239 entries, 0 to 238
Series name: c_idx
Non-Null Count  Dtype 
---

In [22]:
def mean_pooling_with_token_embs(token_embeddings, attention_mask):
    """
    Average the output embeddings using the attention mask 
    to ignore certain tokens.
    """
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

class TransModel(torch.nn.Module):
    """
    Wrapper. Gets tokenized features and returns the output logits for whether the token and content are matching.
    """
    def __init__(self, trans_model_name):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(trans_model_name)
    
    def get_transformer_output(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids = input_ids, attention_mask=attention_mask)
        pooled_embeddings=mean_pooling_with_token_embs(outputs[0], attention_mask)
        return pooled_embeddings

    def forward(self, features):
        # Grab embeddings from transformer
        embs = self.get_transformer_output(
            features[:,0,:],
            features[:,1,:]
        )
        return embs



In [23]:
class CustomDS(torch.utils.data.Dataset):
    def __init__(self, tensor):
        self.topic               = tensor[0]
        self.content             = tensor[1]
            
    def __len__(self):
        return self.topic.size(dim=0)
    
    def __getitem__(self, item):
        
        topic               = self.topic[item]
        content             = self.content[item]


        return topic, content

In [24]:
t_cols = [col for col in train_tokenized_df.columns if col.startswith('t_')]
c_cols = [col for col in train_tokenized_df.columns if col.startswith('c_')]
train_ds = CustomDS(
                    (torch.tensor(train_tokenized_df[t_cols].values.tolist()),
                     torch.tensor(train_tokenized_df[c_cols].values.tolist())
                     )
                    )


/tmp/ipykernel_712321/629240054.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  (torch.tensor(train_tokenized_df[t_cols].values.tolist()),


In [25]:
val_data = (torch.tensor(val_topics_tokenized_df[t_cols].values.tolist()),
            torch.tensor(val_contents_tokenized_df[c_cols].values.tolist())
            ,val_corr_df
            )

In [26]:
try: t_model
except NameError: t_model = None
if t_model is not None:
  del t_model
report_gpu()
#BATCH_SIZE = 760 # 650 works for A100, 20 works good for training for TESTING_SUBSET
if TESTING_SUBSET:
    # 20 works good for training for TESTING_SUBSET
    BATCH_SIZE = 20
else:
    BATCH_SIZE = 580
    # 1100 works good for minilm
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers = 2)

GPU:0
process     700802 uses     3967.000 MB GPU memory
process     711585 uses     1243.000 MB GPU memory


In [27]:
# Add option to load pre-existing model
LOAD_CHECKPOINT_MODEL = False
SAVE_BEST_MODEL = False
DRY_RUN = False
TRAIN_PATIENCE = 3
k_vals = [5,10, 50]
METRIC = 'recall'
METRIC_KEY = 10
MODEL_FILENAME_PREFIX = f'One_trans_distiluse_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_{BATCH_SIZE}'
SAVE_MODEL_DIR = CFG.SAVE_MODEL_DIR
NUM_FEATURES = 3
FEATURE_EMB_DIM = 384
OUTPUT_EMB_DIM = 512
trans_model = TransModel( trans_model_name = TRANS_MODEL)
optimizer = torch.optim.Adam((list(trans_model.parameters())),
                              lr=2e-4,
                             #weight_decay = 1e-4 # L2 regularization, 1e-4 is a good value
                             )
model_dict = {'trans_model': trans_model, 'optimizer': optimizer}
if LOAD_CHECKPOINT_MODEL:
    MODEL_FILENAME = SAVE_MODEL_DIR + 'One_trans_distiluse_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_1100.last_model.pth'
    # t_model, optimizer = load_model(t_model, optimizer = None, filename = MODEL_FILENAME)
    # We'll use a fresh optimizer since we are loading ArcFace pre-trained model.
    # optimizer = torch.optim.Adam((list(t_model.parameters())),
    #                           lr=2e-4,
    #                          #weight_decay = 1e-4 # L2 regularization, 1e-4 is a good value
    #                          )
    # train_scores = []
    # val_scores = []
    # epoch_count = 0
    # MODEL_FILENAME = None
    load_model( model_dict, filename = MODEL_FILENAME)
    train_data_dict = load_dict(SAVE_MODEL_DIR + f'{MODEL_FILENAME_PREFIX}.train.txt')
    train_scores, val_scores = train_data_dict['train_scores'], train_data_dict['val_scores']
    epoch_count = len(val_scores)
else:
    train_scores = []
    val_scores = []
    epoch_count = 0
    MODEL_FILENAME = None

trans_model = trans_model.to(device)
# criterion = nn.BCEWithLogitsLoss()
criterion = MultipleNegativesRankingLoss()



In [28]:
NUM_EPOCHS = 15

DISABLE_TQDM = False
if (DISABLE_TQDM):
    tqdm.__init__ = partialmethod(tqdm.__init__, disable=DISABLE_TQDM)

scaler = torch.cuda.amp.GradScaler()

pt('Starting Training')
trans_model.train()
#for epoch in tqdm(range(EPOCHS), desc="Epochs"):
for epoch in range(NUM_EPOCHS):
    # We set the below in case it is frozen while doing eval
    pt(f"***** Epoch {epoch_count} *****")
    train_loss = 0
    #train_acc = 0
    #train_f2_score = 0
    for (topic, content) in tqdm(train_loader, desc="Training"):
        topic   = topic.to(device)
        content = content.to(device)

        optimizer.zero_grad()

        with torch.autocast(device_type='cuda', dtype=torch.float16):
            t_emb = trans_model(topic)
            c_emb = trans_model(content)
            loss = criterion(t_emb, c_emb)

        # loss.backward()
        # optimizer.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
       
        train_loss += loss.item()
    train_loss = round(train_loss/len(train_loader), 5)
    pt(f'Train : | Loss: {train_loss:.5f} ')
    train_scores.append({'loss':train_loss})
    recalls  = compute_recall_at_k_for_two_trans(trans_model, trans_model, device, val_data[0], val_data[1], val_data[2], k_vals = k_vals, batch_size = BATCH_SIZE)
    recall_str = '\nValidation:  ' + ''.join([f'| Recall@{k}: {recalls[k]:.3f}' for k in k_vals])
    pt(recall_str)
    val_scores.append({'recall':recalls})
    if SAVE_BEST_MODEL:
        MODEL_FILENAME = save_best_model( 
                                        model_dict = model_dict,
                                        old_model_filename = MODEL_FILENAME,
                                        model_filename_prefix = MODEL_FILENAME_PREFIX,
                                        model_path = SAVE_MODEL_DIR,
                                        save_model_fn = save_model,
                                        train_scores = train_scores,
                                        val_scores = val_scores, 
                                        metric = METRIC, metric_key = METRIC_KEY,
                                        dry_run = DRY_RUN
                                        )
    save_model(model_dict, SAVE_MODEL_DIR + f"{MODEL_FILENAME_PREFIX}.last_model.pth")
    epoch_count += 1
    save_dict({'train_scores':train_scores, 'val_scores':val_scores}, SAVE_MODEL_DIR + f'{MODEL_FILENAME_PREFIX}.train.txt')
    if not check_improvement(val_scores, metric = METRIC, metric_key = METRIC_KEY, patience = TRAIN_PATIENCE):
        pt(f'Exiting training since no improvement in the last {TRAIN_PATIENCE} epochs')
        break


2023-03-30 09:57:03.548836 : Starting Training
2023-03-30 09:57:03.549403 : ***** Epoch 0 *****


Training:   0% 0/50 [00:00<?, ?it/s]

Training:   2% 1/50 [00:00<00:26,  1.84it/s]

Training:   4% 2/50 [00:00<00:15,  3.12it/s]

Training:   6% 3/50 [00:00<00:11,  4.14it/s]

...

 42% 5/12 [00:00<00:00, 21.69it/s]

 83% 10/12 [00:00<00:00, 30.47it/s]

100% 12/12 [00:00<00:00, 27.81it/s]

2023-03-30 09:57:58.563485 : 
Validation:  | Recall@5: 0.338| Recall@10: 0.417| Recall@50: 0.569


2023-03-30 09:58:01.724693 : Exiting training since no improvement in the last 3 epochs


In [29]:
if IN_COLAB:
    from google.colab import runtime
    runtime.unassign()